In [543]:
using Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Plots")
#Pkg.add("Statistics")
#Pkg.add("Distributions")
#Pkg.add("Gurobi_jll")
#import Gurobi_jll
#Pkg.add("Gurobi")
#Pkg.build("Gurobi")
#Pkg.add("CairoMakie")
Pkg.add("JSON")

    Updating registry at `C:\Users\djzts\.julia\registries\General.toml`
   Resolving package versions...
    Updating `C:\Users\djzts\.julia\environments\v1.10\Project.toml`
  [682c06a0] + JSON v0.21.4
  No Changes to `C:\Users\djzts\.julia\environments\v1.10\Manifest.toml`


In [544]:
#key = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
#run(`$(Gurobi_jll.grbgetkey()) $key`)
#Pkg.add("Gurobi")
using JSON

In [217]:
using JuMP, Gurobi
using GLPK
#using Plots
using Statistics
using Distributions
using Random

#Data center nodes
#DGX1_power = 3.5 # kWatts
#DGX1_workload = 173 # TFLOPS

#DGX2_power = 10 # kWatts
#DGX2_workload = 2e3 # TFLOPS

#DGXA100_power = 6.5 # kWatts
#DGXA100_workload = 5e3 # TFLOPS

#DGXA100_station_power = 1.5 # kWatts
#DGXA100_station_workload = 1.3e3 # TFLOPS

#DGXB200_power = 14.3 # kWatts
#DGXB200_workload = 72e3 # TFLOPS

#DGXH100_power = 10.2 # kWatts
#DGXH100_workload = 32e3 # TFLOPS


In [552]:
#set up 
node_numbers = 10
Job_numbers = 3 + 30
T_range = 20

#Node_power_spec = Dict{Symbol, Float64}(:DGX1 => 3.5, :DGX2 => 10, :DGXA100 => 6.5, :DGXA100s => 1.5, :DGXA200 => 14.3, :DGXH100 => 10.2)
#Node_compute_spec = Dict{Symbol, Float64}(:DGX1 => 173, :DGX2 => 2e3, :DGXA100 => 5e3, :DGXA100s => 1.3e3, :DGXA200 => 72e3, :DGXH100 => 32e3)

Node_power_spec = Dict{Symbol, Float64}(:DGX1 => 350, :DGX2 => 100, :DGXA100 => 26, :DGXA100s => 15, :DGXA200 => 14.3, :DGXH100 => 10.2)
Node_compute_spec = Dict{Symbol, Float64}(:DGX1 => 17.3e3, :DGX2 => 20e3, :DGXA100 => 20e3, :DGXA100s => 13e3, :DGXA200 => 72e3, :DGXH100 => 32e3)
Node_Types = collect(keys(Node_power_spec))
idle_rate = 0.2 # https://web.eecs.umich.edu/~twenisch/papers/asplos09.pdf

#node_type = rand(Node_Types, node_numbers) # save!
json_list_str = open("node_type.json", "r") do file
    read(file, String)
end
node_type = JSON.parse(json_list_str)

#println(node_type)

Node_power_dic = Dict{Symbol, Float64}()
for (n, node_) in enumerate(node_type)
    key = Symbol(n)
    node_symbol = Symbol(node_)
    node_type[n] = node_symbol
    Node_power_dic[key] = Node_power_spec[node_symbol]
end

display(Node_power_dic);
display(Node_Types);
println(node_type)



Dict{Symbol, Float64} with 10 entries:
  Symbol("9")  => 350.0
  Symbol("6")  => 26.0
  Symbol("1")  => 10.2
  Symbol("8")  => 15.0
  Symbol("7")  => 15.0
  Symbol("2")  => 10.2
  Symbol("10") => 14.3
  Symbol("4")  => 14.3
  Symbol("3")  => 100.0
  Symbol("5")  => 26.0

6-element Vector{Symbol}:
 :DGX1
 :DGXA200
 :DGX2
 :DGXA100
 :DGXA100s
 :DGXH100

Any[:DGXH100, :DGXH100, :DGX2, :DGXA200, :DGXA100, :DGXA100, :DGXA100s, :DGXA100s, :DGX1, :DGXA200]


97

In [595]:
working_jobs = [:wj1, :wj2, :wj3]
working_jobs_times = Dict{Symbol, Int}(:wj1 => 2, :wj2 => 3, :wj3 => 4)
working_jobs_node = Dict{Symbol, Int}(:wj1 => 1, :wj2 => 2, :wj3 => 3)
working_jobs_Reward = Dict{Symbol, Float64}(:wj1 => 5e3, :wj2 => 6e3, :wj3 => 7e3)
working_jobs_Loss = Dict{Symbol, Float64}(:wj1 => 4e3, :wj2 => 4e3, :wj3 => 5e3)
working_jobs_Datasize = Dict{Symbol, Float64}(:wj1 => 6e3, :wj2 => 9e3, :wj3 => 12e3)
#working_jobs_Nparam = Dict{Symbol, Int}(:wj1 => 100, :wj2 => 500, :wj3 => 1e3)  #unit Million 6N is r
working_jobs_TDP_percentage = Dict{Symbol, Float64}(:wj1 => 0.3, :wj2 => 0.5, :wj3 => 0.75) # utilization_rates between 0.3 ~ 0.75 
working_jobs_data_size = Dict{Symbol, Float64}(:wj1 => 6e3, :wj2 => 9e3, :wj3 => 14e3)  # unit in GB

job_pool_number = Job_numbers - length(working_jobs)
job_pool_times = Dict{Symbol, Int}()
job_pool_Reward = Dict{Symbol, Float64}()
job_pool_Loss = Dict{Symbol, Float64}()
job_pool_standard_times = Dict{Symbol, Float64}()
job_pool_time_UB = Dict{Symbol, Int}()
job_pool_times = Array{Float64,2}(zeros(job_pool_number, node_numbers))
job_pool_data_size = Dict{Symbol, Float64}()  # unit in GB
job_pool_TDP_percentage = Dict{Symbol, Float64}() 

node_T_su = 2
node_T_sd = 2

indices = []
json_list_str = open("indices.json", "r") do file
    read(file, String)
end
indices_ = JSON.parse(json_list_str)
for item in indices_
push!(indices, Int(item))
end

json_list_str = open("job_pool_standard_times.json", "r") do file
    read(file, String)
end
job_pool_standard_times_ = JSON.parse(json_list_str)
for item in job_pool_standard_times_
    #display(item)
    job_pool_standard_times[Symbol(item[1])] = Int(item[2])
end


json_list_str = open("job_pool_Reward.json", "r") do file
    read(file, String)
end
job_pool_Reward_ = JSON.parse(json_list_str)
for item in job_pool_Reward_
    #display(item)
    job_pool_Reward[Symbol(item[1])] = Float64(item[2])
end

json_list_str = open("job_pool_Loss.json", "r") do file
    read(file, String)
end
job_pool_Loss_ = JSON.parse(json_list_str)
for item in job_pool_Loss_
    #display(item)
    job_pool_Loss[Symbol(item[1])] = Float64(item[2])
end

json_list_str = open("job_pool_data_size.json", "r") do file
    read(file, String)
end
job_pool_data_size_ = JSON.parse(json_list_str)
for item in job_pool_data_size_
    #display(item)
    job_pool_data_size[Symbol(item[1])] = Float64(item[2])
end

json_list_str = open("job_pool_TDP_percentage.json", "r") do file
    read(file, String)
end
job_pool_TDP_percentage_ = JSON.parse(json_list_str)
for item in job_pool_TDP_percentage_
    #display(item)
    job_pool_TDP_percentage[Symbol(item[1])] = Float64(item[2])
end

json_list_str = open("job_pool_times.json", "r") do file
    read(file, String)
end
job_pool_times_ = JSON.parse(json_list_str)
for (i, vec) in enumerate(job_pool_times_)
    #display(vec)
    for (j, item) in enumerate(vec)
    #display(item)
        job_pool_times[j,i] = Int(item)
    end
end

json_list_str = open("job_pool_time_UB.json", "r") do file
    read(file, String)
end
job_pool_time_UB_ = JSON.parse(json_list_str)
for item in job_pool_time_UB_
    #display(item)
    job_pool_time_UB[Symbol(item[1])] = Int(item[2])
end


"""
compute_ref = :DGX2
# Define the length of the vector and the number of 1's
number_of_ones = Int(floor(0.3 * job_pool_number))

# Generate a vector of 0's and set exactly number_of_ones positions to 1
time_sense_flag = zeros(Int, job_pool_number)
indices = randperm(job_pool_number)[1:number_of_ones]
time_sense_flag[indices] .= 1

for i in 1:job_pool_number

    # Define the probabilities
    probabilities = [0.2, 0.8]

    # Generate a random number 0 or 1 based on the defined probabilities
    #time_sense_flag = rand([0, 1], 1; p=probabilities)[1]
    #time_sense_flag  = rand(Bool) ? 1 : 0

    if time_sense_flag[i] == 1
        key = Symbol("tjp", i)
    else
        key = Symbol("jp", i)
    end
    job_pool_standard_times[key] = rand(5:15)  # take DGX2 as the stanard unit cumpyte time from 20 intervals to 60 intervals
    job_pool_Reward[key]= job_pool_standard_times[key] * round(rand(Uniform(0.8, 1.5)), digits=2)* 1e3
    job_pool_Loss[key]= job_pool_Reward[key] * round(rand(Uniform(1.2, 1.55)), digits=2)
    job_pool_data_size[key]= job_pool_Reward[key] * round(rand(Uniform(0.7, 0.95)), digits=2)
    job_pool_TDP_percentage[key] = round(rand(Uniform(0.4, 0.95)), digits=3)

    for (index, type) in enumerate(node_type)
        #print(index)
        ratio = Node_compute_spec[compute_ref] ./ Node_compute_spec[type]
        job_pool_times[i,index] = ceil(job_pool_standard_times[key] * ratio)
    end

    if time_sense_flag[i] == 1
        job_pool_time_UB[key] = Int(rand( job_pool_standard_times[key] + 2: T_range-1))
    else
        job_pool_time_UB[key] = T_range
    end

end

"""



#display(job_pool_times)

job_collection = sort!(vcat(working_jobs, collect(keys(job_pool_Reward))))
Reward_collection = merge(working_jobs_Reward, job_pool_Reward)
Loss_collection = merge(working_jobs_Loss, job_pool_Loss)
Data_size_collection = merge(working_jobs_Datasize, job_pool_data_size)
TDP_percentage = merge(working_jobs_TDP_percentage, job_pool_TDP_percentage)

job_node_init_state = Array{Float64,2}(zeros(Job_numbers, node_numbers))


Transmit_fee = 0.07 #0.07 #$/GB #https://azure.microsoft.com/en-us/pricing/details/bandwidth/
Transmit_Bound = 1e3 #G/s 
Job_Transmit_Bound = 5e2 # G/s
Transmit_power = 5.12 #kWh/GB
C_transmit_job =  Dict{Symbol, Float64}()  #Array{Float64}(zeros(length(job_collection)))

for (j, item) in enumerate(job_collection)
    #print(item)
    C_transmit_job[item] = Data_size_collection[item] * Transmit_fee
    if isa(item, Symbol) && startswith(string(item), "wj")
        #print(j)
        #println(working_jobs_node[item])
        job_node_init_state[j, working_jobs_node[item]] = 1
    end
end
#display(job_pool_times)

#display(job_pool_Reward)
#display(C_transmit_job)
#job_pool_times

In [220]:
job_pool_time_UB

Dict{Symbol, Int64} with 30 entries:
  :tjp23 => 18
  :jp24  => 20
  :tjp22 => 13
  :jp3   => 20
  :jp16  => 20
  :tjp14 => 16
  :jp28  => 20
  :jp18  => 20
  :tjp26 => 9
  :jp5   => 20
  :tjp27 => 16
  :jp9   => 20
  :tjp2  => 18
  :jp8   => 20
  :jp13  => 20
  :jp20  => 20
  :tjp30 => 19
  :jp10  => 20
  :jp11  => 20
  ⋮      => ⋮

In [221]:
Data_size_collection

Dict{Symbol, Float64} with 33 entries:
  :tjp23 => 11553.6
  :wj3   => 12000.0
  :jp24  => 11161.7
  :jp19  => 14563.5
  :tjp22 => 5928.0
  :jp3   => 13112.0
  :jp16  => 18081.0
  :tjp14 => 18233.6
  :jp28  => 8658.0
  :jp18  => 10276.5
  :tjp26 => 7867.8
  :wj2   => 9000.0
  :jp5   => 6561.0
  :tjp27 => 4120.0
  :jp9   => 7224.0
  :tjp2  => 17640.0
  :jp8   => 5532.8
  :jp13  => 17656.5
  :jp20  => 11484.0
  ⋮      => ⋮

In [222]:
Job_Transmit_Bound

500.0

In [223]:
# Define the Genthm_info array

    #1      2	    3	    4	    5	    6	    7	   8	9   10      11  12	        13	        14?    				
    #bus    reg	    pmax	pmin	ru	    rd	    su	   sd	tup	tdown	u0  ocost_st	a	        b(*p)	c(*p^2)	 ##x0	ccost	life	p_fo	SO2	CO2	

Genthm_info = [
    1       1       50      20      30      30     5*100   0    2   1       1 0 0.5*10     20      0       0       0       0       0       0       0;
    2       1       650     250     200     200    10*100  0    3   1       0 0 1.5*10     15      0       0       0       0       0       0       0;
    3       1       3450    1800    1800    1800   30*100  0    3   1       0 0 5.0*10     17.5    0       0       0       0       0       0       0;
]

# MVA baseKV
SBase = 100 #VA

# Amount of Generator and the indices they possess
G_I = size(Genthm_info, 1)
G_I_list = Genthm_info[:, 1]

# Generator Power limit
# P_gen_max and P_gen_min: Maximum and Minimum Active Power of Generators in p.u.
P_UB = Genthm_info[:, 3]
P_LB = Genthm_info[:, 4]

# Ramping up (operating/cold start) limitation for unit i. [5]
RU_O_UB = P_UB - P_LB
RU_CS_UB = P_UB - P_LB

# Ramping down (operating/hot close) limitation for unit i. [6 ]
RD_O_UB = P_UB - P_LB
RD_HS_UB = P_UB - P_LB

# Startup and shut down Cost SU_{i,t} SD_{i,t}
G_ST_cost = Genthm_info[:, 7]
G_SD_cost = Genthm_info[:, 8]

# Minimum on and off time periods for unit i.
UT = Int.(Genthm_info[:, 9])
DT = Int.(Genthm_info[:, 10])

# Gen initial status code 0(off) / 1(on) 
u_0 = Genthm_info[:, 11]

# The time intervals that the generator has turned ON/OFF(V_{i,0}/S_{i,0}) after 0. (Only for One Bin Model)
# This param is user-defined, which is not retrieved from
G_Status_on = UT
G_Status_off = DT

# Operation cost coefficient
cost_st = Genthm_info[:, 12]

# Start up cost coefficient, fix cost for u $Gamma$
cost_fix = Genthm_info[:, 13]

# Start up cost coefficient, fix cost for p $Beta$
costOf = Genthm_info[:, 14]

Gen_thm_node_num = size(Genthm_info)[1];

In [604]:
#Data Center setup
# Total Data center zones
I_zone_wide = 2
I_zone_length = 2
I_zone = I_zone_wide * I_zone_length

# time interval length
delta_t = 60 # minutes

# Real time grid electricity price
p_eg = collect(1:T_range) * 0.1492

# Battery electricity charging upper bound
char_b_upperbound = 60 * delta_t / 60

# Battery electricity discharging upper bound
dis_b_upperbound = 90 * delta_t / 60

# Battery electricity capacity
Battery_capacity = 250 # unit kWh

# Battery initial energy
#Battery_capacity_init = 0.5 * Battery_capacity # unit kWh

# Battery electricity capacity upper and lower alert
Battery_capacity_upbound = 0.85 * Battery_capacity
Battery_capacity_lowbound = 0.15 * Battery_capacity

# Battery charging efficiency
eta_char_b = 0.9

# Battery discharging efficiency
eta_dischar_b = 1 / 0.85

# Zone temperature initialization
T_initial_upperbound = 33
T_initial_lowerbound = 25


"""
random_choice = 1
if random_choice == 0
    T_zone_initial = [31 30 29; 32 26 26; 30 24 27]
    # Outside air temperature
    T_out = [30, 34, 32] # unit K
else
    #println()
    T_zone_initial = rand(T_initial_lowerbound:T_initial_upperbound-7, I_zone_wide, I_zone_length)
    # Outside air temperature
    T_out = rand(27:36, T_range) # unit Celsius
end

# Zone space
if random_choice == 0
    Zone_space = [1020 950 950; 1010 1060 1040; 900 1050 930]
else
    Zone_space = rand(900:1100, I_zone_wide, I_zone_length)
end

random_choice_solar = 1
# Solar electricity production
if random_choice_solar == 0
    e_solar = [65, 85, 40] * delta_t / 60 # kWh
else
    e_solar = rand(40:150, T_range) * delta_t / 60
end
"""


# Read from the file
json_vector_str = open("T_out.json", "r") do file
    read(file, String)
end
# Convert JSON string back to a Julia vector
T_out = JSON.parse(json_vector_str)

# Read from the file
json_vector_str = open("T_zone_initial.json", "r") do file
    read(file, String)
end
# Convert JSON string back to a Julia vector
T_zone_initial = JSON.parse(json_vector_str)


# Read from the file
json_vector_str = open("Zone_space.json", "r") do file
    read(file, String)
end
# Convert JSON string back to a Julia vector
Zone_space = JSON.parse(json_vector_str)

# Read from the file
json_vector_str = open("e_solar.json", "r") do file
    read(file, String)
end
# Convert JSON string back to a Julia vector
e_solar = JSON.parse(json_vector_str)

# Read from the file
json_vector_str = open("M_chiller_rated.json", "r") do file
    read(file, String)
end
# Convert JSON string back to a Julia vector
M_chiller_rated = JSON.parse(json_vector_str)

# Read from the file
json_vector_str = open("M_tower_rated.json", "r") do file
    read(file, String)
end
# Convert JSON string back to a Julia vector
M_tower_rated = JSON.parse(json_vector_str);

DC_height = 6 # meters

# Server electricity consumption
#e_server = delta_t / 60 * 2.5 * ones(T_range) * sum(Zone_space) # kWh

# Zone air
R_air_ij = 0.062 # unit m.K/W

# Air heat capacity in room i
#C_air =  718 * 1.225 * 100 * np.ones(I_zone) # unit J/K  rho = 1.225 kg/m3

# AC wind speed
v_sup = ones(T_range) * 1 # unit m/s

# Air mass flow into the room i
m_air_in = 4 * kron(reshape(v_sup, 1, :), ones(I_zone, 1)) * 1.2 * delta_t * 60 # unit m2 * m/s * Kg/m^3 * s

# Specific heat capacity of air in room i
c_s_air = 0.718 # unit [KJ/( kg · K )]

# Internal heat generation of server radiated to room i
#theta_heat = delta_t / 60 * 2.5 * kron(ones(1, T_range), reshape(Zone_space, :, 1)) # unit W/K

# Outside wind speed
v_out = ones(T_range) * 0.5 # unit m/s

# AC wind power
beta_sup = 5 # unit kWh/(m/s)
e_sup = beta_sup * v_sup # unit kWh

# Ventilation air flow rate lower bound
v_vent_lowerbound = ones(T_range) .* (0.8 .- v_out) # unit m/s

# Heat capacity of air.
C_air_room = 0.718 * 1.2 * Zone_space * DC_height # unit [KJ/( kg · K )] * kg/m3 * m3 = KJ/K
C_air_room_flat = vec(C_air_room)

# Heat capacity of water.
C_water = 4.182 # unit [KJ/( kg · K )]

# Room air temperature upper bound
T_datacenter_upperbound = ones(I_zone, T_range) * 24
T_datacenter_upperbound_plus = ones(I_zone, T_range) * 30

# Room air temperature lower bound
T_datacenter_lowerbound = ones(I_zone, T_range) * 15

# Room AC temperature upper bound
T_AC_upperbound = ones(I_zone, T_range) * 33

# Room AC temperature lower bound
T_AC_lowerbound = ones(I_zone, T_range) * 10

# Chiller water temperature supply
T_chiller_water_supply = ones(T_range) * 20

# Chiller water temperature return
T_chiller_water_return = ones(T_range) * 30

# Condense water temperature supply
T_tower_water_supply = T_out

# Condense water temperature return
T_tower_water_return = ones(T_range) * 41

# Ventilation air flow rate start-up power
beta_vent_0 = 2 # unit kWh
beta_vent_1 = 0.2 # unit kWh s/m

const_vent = beta_vent_0 .+ beta_vent_1 * v_vent_lowerbound .* delta_t .* 60

# Pump water flow rate power
beta_pump_0 = 5 # unit kWh
beta_pump_1 = 10 # unit kWh/(kg)

# Chiller amount
J_chiller = 4

# Tower amount
J_tower = 5

# Chiller rated water flow rate
"""
random_choice_M = 0
if random_choice_M == 0
    M_chiller_rated = [60, 100, 140, 180] # unit kg/t
    M_tower_rated = [50, 75, 100, 125, 150] # unit kg/t
else
    M_chiller_rated = rand(60:160, J_chiller) # unit kg/t
    M_tower_rated = rand(50:150, J_tower) # unit kg/t
end
"""

# Chiller rated power
beta_chiller_0 = 4 * ones(J_chiller) # unit kWh
beta_chiller_1 = 7 * ones(J_chiller) # unit kWh/(kg)

# Tower rated power
beta_tower_0 = 6 * ones(J_tower) # unit kWh
beta_tower_1 = 5 * ones(J_tower) # unit kWh/(kg)

# Weight of each zone in data center
Chi = ones(I_zone) * (1 / I_zone)

# Adjacent zone map (default)
# 4|5|6|
# 1|2|3|
adjacent_zone_map = zeros(I_zone, I_zone)
for zone in 1:I_zone
    zone_x = div(zone - 1, I_zone_length) + 1
    zone_y = mod(zone - 1, I_zone_length) + 1
    
    #println(zone_x, zone_y)

    adjacent_zone_x = [zone_x - 1, zone_x + 1]
    adjacent_zone_y = [zone_y - 1, zone_y + 1]

    #println(adjacent_zone_x, adjacent_zone_y)
    
    adjacent_zone_x_update = adjacent_zone_x[(adjacent_zone_x .>= 1) .& (adjacent_zone_x .<= I_zone_wide)]
    adjacent_zone_y_update = adjacent_zone_y[(adjacent_zone_y .>= 1) .& (adjacent_zone_y .<= I_zone_length)]

    #println(adjacent_zone_x_update, adjacent_zone_y_update)

    adjacent_zone_map[zone, (adjacent_zone_x_update .- 1) .* I_zone_length .+ zone_y ] .= 1
    adjacent_zone_map[zone, (zone_x - 1) .* I_zone_length .+ adjacent_zone_y_update ] .= 1
end

K_chiller = c_s_air ./ ((T_chiller_water_return - T_chiller_water_supply) * C_water)

#println(T_tower_water_supply)
K_tower = c_s_air ./ ((T_tower_water_return - T_tower_water_supply) * C_water)

# Constant pump
#const_pump = beta_pump_0 .+ beta_pump_1 .* transpose(K_chiller) .* (T_out .* v_out)
const_pump = beta_pump_0 .+ beta_pump_1 .* K_chiller .* (T_out .* v_out);

In [632]:
# Ensure random seed for reproducibility
Random.seed!(1234)

Zone_node_type = Dict{Int, Vector{Symbol}}()
Zone_node_indices = Dict{Int, Vector{Int}}()


# Read from the file
json_vector_str = open("Zone_node_type.json", "r") do file
    read(file, String)
end
# Convert JSON string back to a Julia vector
Zone_node_type_ = JSON.parse(json_vector_str);
for item in Zone_node_type_
    #println(item[1], item[2])
    Zone_node_type[parse(Int, item[1])] = [Symbol(thing) for thing in item[2]]
end
Zone_node_type


# Read from the file
json_vector_str = open("Zone_node_indices.json", "r") do file
    read(file, String)
end
# Convert JSON string back to a Julia vector
Zone_node_indices_ = JSON.parse(json_vector_str);
for item in Zone_node_indices_
    #println(item[1], item[2])
    Zone_node_indices[parse(Int, item[1])] = [Int(thing) for thing in item[2]]
end
println(Zone_node_indices)

"""
# Randomly shuffle the indices of Node_Types
indices = shuffle(1:length(node_type))

#display(indices)
#display(Node_power_spec)

# Initialize the bins as an array of arrays

bins = [[] for _ in 1:I_zone]
bins_indices = [[] for _ in 1:I_zone]

# Distribute one element to each bin to ensure each bin has at least one element
for i in 1:I_zone
    push!(bins[i], node_type[indices[i]])
    push!(bins_indices[i], indices[i])
end

for i in (I_zone+1):length(node_type)
    rand_seed = rand(1:I_zone)
    push!(bins[rand_seed], node_type[indices[i]])
    push!(bins_indices[rand_seed], indices[i])
end

for i in 1:I_zone
    Zone_node_type[i] = bins[i]
    Zone_node_indices[i] = bins_indices[i]
end
"""

Dict(4 => [6], 2 => [9, 1, 7], 3 => [2, 10, 3, 4], 1 => [8, 5])


"# Randomly shuffle the indices of Node_Types\nindices = shuffle(1:length(node_type))\n\n#display(indices)\n#display(Node_power_spec)\n\n# Initialize the bins as an array of arrays\n\nbins = [[] for _ in 1:I_zone]\nbins_indices = [[] for _ in 1:I_zone]\n\n# Distribute one element to" ⋯ 156 bytes ⋯ "\n\nfor i in (I_zone+1):length(node_type)\n    rand_seed = rand(1:I_zone)\n    push!(bins[rand_seed], node_type[indices[i]])\n    push!(bins_indices[rand_seed], indices[i])\nend\n\nfor i in 1:I_zone\n    Zone_node_type[i] = bins[i]\n    Zone_node_indices[i] = bins_indices[i]\nend\n"

In [633]:
"""
# Convert to JSON strings
json_dict1 = JSON.json(Zone_node_type)
json_dict2 = JSON.json(Zone_node_indices)

# Save to files
open("Zone_node_indices.json", "w") do file
    write(file, json_dict2)
end
"""

"# Convert to JSON strings\njson_dict1 = JSON.json(Zone_node_type)\njson_dict2 = JSON.json(Zone_node_indices)\n\n# Save to files\nopen(\"Zone_node_indices.json\", \"w\") do file\n    write(file, json_dict2)\nend\n"

In [634]:
display(Zone_node_indices)
display(Zone_node_type)

Dict{Int64, Vector{Int64}} with 4 entries:
  4 => [6]
  2 => [9, 1, 7]
  3 => [2, 10, 3, 4]
  1 => [8, 5]

Dict{Int64, Vector{Symbol}} with 4 entries:
  4 => [:DGXA100]
  2 => [:DGX1, :DGXH100, :DGXA100s]
  3 => [:DGXH100, :DGXA200, :DGX2, :DGXA200]
  1 => [:DGXA100s, :DGXA100]